In [ ]:
import pandas as pd

data = pd.read_csv('county-data.csv')
county_df = pd.DataFrame(data)

data = pd.read_csv('municipal-data.csv')
muni_df = pd.DataFrame(data)


In [ ]:
columns_to_drop = ['RANK', 'SUMLEV', 'PLACE', 'COUSUB', 'CONCIT', 'PRIMGEO_FLAG', 'FUNCSTAT', 'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2022', 'Type', 'UID', 'Population Sum', '% of Population', 'Total']
county_df = county_df.drop(columns=columns_to_drop)

print(county_df)

In [ ]:
columns_to_drop = ['RANK', 'Population Sum', '% of Population']

muni_df = muni_df.drop(columns=columns_to_drop)\
                .rename(columns={'NAME': 'MUNI NAME'})

print(muni_df)

### Merge with County List

In [ ]:
merged_data = pd.merge(muni_df, county_df, how='left', on=['STATE', 'COUNTY'])

merged_data['COUNTY NAME'] = merged_data['NAME']

print(merged_data)


In [ ]:
columns_to_drop = ['STNAME_y', 'IS COUNTY', 'STATESTATE', 'NAME', 'COUNTY DISTINCTION']

merged_data = merged_data.drop(columns=columns_to_drop)\
                        .rename(columns={'STNAME_x': 'STNAME'})

print(muni_df)

In [ ]:
types_to_drop = ['government', 'Region', 'Parish', 'UT', 'Bow', 'CDP']
merged_data = merged_data[~merged_data['TYPE'].isin(types_to_drop)]


In [ ]:
merged_data.drop_duplicates(subset='UID Full', keep=False, inplace=True)

In [ ]:
merged_data.to_csv('updated-municipalities.csv', index=False)

### Merge with Census Place FIPS Code List

In [ ]:
census_df = pd.read_csv('census-place-list.csv', sep='|')

census_df = census_df.astype(str)

# Replace pipe delimiters with commas
census_df = census_df.applymap(lambda x: x.replace('|', ','))

# Write the modified data back to a CSV file
census_df.to_csv('census-place-reformatted.csv', index=False)
census_df = pd.read_csv('census-place-reformatted.csv')

In [ ]:
columns_to_drop = ['STATE', 'PLACENAME', 'TYPE', 'CLASSFP', 'FUNCSTAT', 'PLACENS']

census_df = census_df.drop(columns=columns_to_drop)\
                    .rename(columns={'PLACEFP':'PLACE', 'STATEFP': 'STATE'})

print(census_df)

In [ ]:
muni_data2 = pd.read_csv('updated-municipalities.csv')
muni_data2.drop_duplicates(subset='UID Full', keep=False, inplace=True)

merged_data = pd.merge(muni_data2, census_df, how='left', on=['STATE', 'PLACE'])

merged_data.rename(columns={'COUNTIES': 'COUNTY NAME'}, inplace=True)

print(merged_data)

In [ ]:
# merged_data.sort_values('IS MULTIPLE', inplace=True, ascending=False)

merged_data.sort_values('POPESTIMATE2022', inplace=True, ascending=False)

merged_data.to_csv('updated-municipalities2.csv', index=False)